In [1]:
sub_list = ['005','006','007','008','009','010','011','012','013','015','017','018','020','022','023','024']
task_list = ['localizer','imagine']
mask_list=['bilateral_hippo','bilateral_oc-temp']

In [10]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
import nibabel as nib
import nilearn
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
from nilearn import image
from nilearn import plotting
from nilearn.plotting import plot_roi
from scipy import stats
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import scipy.io
#from mpi4py import MPI
import os
import pickle 
import time
from scipy.sparse import random
from scipy.stats import zscore
from scipy.stats.stats import pearsonr   
from scipy.spatial.distance import euclidean
from pathlib import Path
from shutil import copyfile
import seaborn as sns
import importlib
import csv

# Import machine learning libraries
import sklearn
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, PredefinedSplit
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, roc_auc_score

%matplotlib inline 
%autosave 5

Autosaving every 5 seconds


In [3]:
from platform import python_version
print('The python version is {}.'.format(python_version()))
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The numpy version is {}.'.format(np.__version__))
print('The nilearn version is {}.'.format(nilearn.__version__))
print('The seaborn version is {}.'.format(sns.__version__))

The python version is 3.9.10.
The scikit-learn version is 1.0.2.
The numpy version is 1.20.3.
The nilearn version is 0.8.1.
The seaborn version is 0.11.2.


In [7]:
# Set printing precision
np.set_printoptions(precision=2, suppress=True)

# load some helper functions
sys.path.insert(0, '/jukebox/norman/karina/adderzip_fMRI/adderzip/code/mainanalysis')

#imagine
import adderzip_utils_imagine
from adderzip_utils_imagine import load_adderzip_stim_labels_imagine,load_adderzip_epi_data, shift_timing, label2TR, mask_data

# load some constants
from adderzip_utils_imagine import adderzip_dir, adderzip_bids_dir, adderzip_label_dict_imagine, adderzip_TR, adderzip_hrf_lag, run_names, run_order_start, n_runs, TRs_run

data_dir='/jukebox/norman/karina/adderzip_fMRI/adderzip/data/mainanalysis/output'

importlib.reload(adderzip_utils_imagine)

task = task_list[1]
print('TASK:', task)
print('LIST OF TASKS:', run_names)
task_index = run_names.index(task)
print('task index:', task_index)
print('')

n_runs_imagine = n_runs[task_index]

TRs_run_imagine=144
TRs_run_extra_imagine=216

trials_run_imagine = 32
trials_run_extra_imagine = 48

print('ROIs = %s' % (mask_list))
print('Labels = %s' % (adderzip_label_dict_imagine))
print('number of runs = %d' % (n_runs_imagine))
print('TRs per run after trimming = %s' % (TRs_run_imagine))
print('TRs per run after trimming = %s' % (TRs_run_extra_imagine))

TASK: imagine
LIST OF TASKS: ['imagine']
task index: 0

ROIs = ['bilateral_hippo', 'bilateral_oc-temp']
Labels = {0: 'Scenes', 1: 'Faces'}
number of runs = 2
TRs per run after trimming = 144
TRs per run after trimming = 216


In [8]:
def getLabel(this_sub,this_run,this_task): 
    
    label_dir = data_dir + '/label_list_%s/labels_run-0%i_%s.csv' % (this_task, this_run, this_sub)
    thisLabel = open(label_dir)
    thisLabel = csv.reader(thisLabel)
    thisLabel = list(thisLabel)
    thisLabel = np.array(thisLabel)
    thisLabel = np.float64(thisLabel)
    
    return thisLabel

def getBold(this_sub,this_run,this_task,this_mask):
    
    bold_dir = data_dir + '/bold_data_%s/%s/bold_run-0%i_%s.csv' % (this_task, this_mask, this_run, this_sub)
    thisBold = open(bold_dir)
    thisBold = csv.reader(thisBold)
    thisBold = list(thisBold)
    thisBold = np.array(thisBold)
    thisBold = np.float64(thisBold)
    
    return thisBold

def getGroup(this_sub,this_run):
    
    group_dir = data_dir + '/itemGroup_imagine/itemGroup_run-0%i_%s.csv' % (this_run, this_sub)
    group = open(group_dir)
    group = csv.reader(group)
    group = list(group)
    group = np.array(group)
    group = np.float64(group)
    #print('group')
    #print(group)
    
    return group

In [22]:
#RSA of generalization vs exceptions
#before & after

imagine_runs = np.array([8,9])

subNum = '005'
eachRun = 0

thisRun = imagine_runs[eachRun]

thisBold = getBold(subNum,thisRun,'imagine','bilateral_hippo')
print('boldshape',np.shape(thisBold))
#thisLabel = getLabel(subNum,thisRun,'imagine')
group = getGroup(subNum,thisRun)
print('group',len(group))

gens = np.where(group[:]==3)[0]
print(len(gens))
excpts = np.where(group[:]==5)[0]
print(len(excpts))

gensBold = thisBold[gens,:]
print(len(gensBold))
excptsBold = thisBold[excpts,:]
print(len(excptsBold))

#gensLabel = thisLabel[gens]
#excptsLabel = thisLabel[excpts]

# compute differentiation correlation
gensExcpts = pearsonr(gensBold, excptsBold)[0]
#first value is Pearson's r, second value is p-value
print(gensExcpts)





boldshape (144, 1950)
group 144
34
33
34
33


ValueError: x and y must have the same length.